<a href="https://colab.research.google.com/github/jjaw89/spring_2025_dl_audio_project/blob/test_run_vocoder/test_vocoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import same packages as the train script in Wave-U-Net-Pytorch
import os
import time

import torch
import numpy as np

# import torch.nn as nn
# from torch.utils.tensorboard import SummaryWriter
# from torch.optim import Adam
# from torch.nn import L1Loss
# from tqdm import tqdm
# from torchsummary import summary
# import torch.optim as optim
# from torch.utils.data import Dataset, DataLoader
# import numpy as np
# # install torchaudio if not already installed
# # ! pip install torchaudio
# import torchaudio

from typing import Tuple, List, Dict, Optional
import librosa
import soundfile as sf
import IPython.display as ipd

from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/git_projects/spring_2025_dl_audio_project

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/git_projects/spring_2025_dl_audio_project


Classical method for phase construction (convert spectrogram to audio)

In [2]:
def convert_to_audio(mel_spectrograms, batch_size = 8, n_fft=2048, hop_length=512, power=2.0, n_iter=32):
    audio_files = []
    sr = 44100

    for i in range(0, len(mel_spectrograms), batch_size):
        batch = mel_spectrograms[i:i+batch_size]
        batch_audio = []

        for mel_spec in batch:
            # Convert Mel spectrogram back to linear spectrogram
            linear_spec = librosa.feature.inverse.mel_to_stft(
                mel_spec, sr=sr, n_fft=n_fft, power=power
            )

            # Apply Griffin-Lim algorithm for phase reconstruction
            audio = librosa.griffinlim(
                linear_spec, hop_length=hop_length, n_iter=n_iter
            )

            batch_audio.append(audio)

        audio_files.extend(batch_audio)
        print(f"Processed batch {i//batch_size + 1} of {(len(mel_spectrograms) + batch_size - 1)//batch_size}")

    return audio_files

In [4]:
audio_file = "/content/drive/MyDrive/git_projects/spring_2025_dl_audio_project_data/LibriSpeech/train-clean-100/19/198/19-198-0000.flac"

In [5]:
# Load audio
y, sr = librosa.load(audio_file, sr=44100)

In [6]:
ipd.display(ipd.Audio(y, rate=44100))

In [7]:
sr = 44100
n_fft = 2048
hop_length = 512
n_mels = 128
batch_size = 1
power = 2.0
n_iter = 32

In [8]:
mel_spec = librosa.feature.melspectrogram(
    y=y,
    sr=44100,
    n_fft=n_fft,
    hop_length=hop_length,
    n_mels=n_mels
)

In [9]:
mel_specs_list = [mel_spec]
reconstructed_list = convert_to_audio(
    mel_specs_list,
    batch_size=batch_size,  # Batch size of 1 for single mel-spec
    n_fft=n_fft,
    hop_length=hop_length,
    power=power,
    n_iter=n_iter
)
# Extract the reconstructed audio from the list
y_reconstructed = reconstructed_list[0]

Processed batch 1 of 1


In [10]:
print("Play reconstructed audio again:")
ipd.display(ipd.Audio(y_reconstructed, rate=44100))

Play reconstructed audio again:


In [24]:
!git status

On branch test_run_vocoder
Your branch is up to date with 'origin/test_run_vocoder'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   GAN_train.ipynb

no changes added to commit (use "git add" and/or "git commit -a")
